# MetImages Tensorboard Visualization

This notebook takes the vector features extracted in MetMuseum_FeatureSearch.ipynb and prepares them to be visualized in Tensorboard. Check tensorboard and tensorflow versions as there might be compatibility issues between versions. My versions are Tensorflow & Tensorboard 1.14.0 

Original Kaggle code: https://www.kaggle.com/pavansanagapati/tensorboard-visualisation-of-fashion-items


Even more original code without indentation problems: https://nanonets.com/blog/how-to-classify-fashion-images-easily-using-convnets/

In [1]:
# Import Numpy for statistical calculations
import numpy as np

# Import Pandas for data manipulation using dataframes
import pandas as pd

# Import Warnings 
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
from IPython.core.display import HTML 

# Import matplotlib Library for data visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from tensorflow.examples.tutorials.mnist import input_data

# Input data files are available in the "data/" directory.
# For example, running this will list the files in the input directory
import os


In [2]:
import pickle

In [3]:
images, pca_features, pca = pickle.load(open(r"D:\Path\to\features_MetFeatures.p", 'rb'))

for img, f in list(zip(images, pca_features))[0:5]:
    print("image: %s, features: %0.2f,%0.2f,%0.2f,%0.2f... "%(img, f[0], f[1], f[2], f[3]))

image: /storage/MetImages/ml4a-guides/utils/Cropped/f00443_Lady Lilith337500.jpg, features: 6.79,-21.71,1.03,0.84... 
image: /storage/MetImages/ml4a-guides/utils/Cropped/f00663_Saint Anthony the Abbot in the Wilderness458967.jpg, features: 12.30,-19.85,3.65,6.67... 
image: /storage/MetImages/ml4a-guides/utils/Cropped/f00093_Block Statue of a Prophet of Montu and Scribe Djedkhonsuefankh, son of Khonsumes and Taat547694.jpg, features: -24.40,23.30,15.20,-16.92... 
image: /storage/MetImages/ml4a-guides/utils/Cropped/f00265_Encyclop die, ou Dictionnaire raisonn  des sciences, des arts et des m tiers591843.jpg, features: 1.29,-6.06,-8.16,-2.99... 
image: /storage/MetImages/ml4a-guides/utils/Cropped/f00850_The Love Song435826.jpg, features: 10.73,-27.55,8.71,4.90... 


In [4]:
imgidx = []
for i in images:
    imgidx.append(i[45:])

In [5]:
X_test = pca_features
Y_test = np.array(imgidx)

In [6]:
X_test.shape[0]

980

### Summary writer

In [7]:
logdir = 'metviz_logs'

# Use this logdir to create a summary writer
summary_writer = tf.summary.FileWriter(logdir)

In [8]:
# Creating the embedding variable with all the images defined above under X_test
embedding_var = tf.Variable(X_test, name='metviz_logs')

In [9]:
embedding_var

<tf.Variable 'metviz_logs:0' shape=(980, 300) dtype=float32_ref>

In [10]:
# Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here I add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(logdir, 'metadata.tsv')

# After constructing the sprite, I need to tell the Embedding Projector where to find it
embedding.sprite.image_path = os.path.join(logdir, 'sprite.png')
embedding.sprite.single_image_dim.extend([28, 28])

In [11]:
# The next line writes a projector_config.pbtxt in the logdir. TensorBoard will read this file during startup.
projector.visualize_embeddings(summary_writer,config)

In [12]:
# Periodically save the model variables in a checkpoint in logdir.
with tf.Session() as sesh:
    sesh.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.save(sesh, os.path.join(logdir, 'model.ckpt'))

After you have made the sprite and metadata files below, paste both of them into the 'metviz_logs' folder. Then go a folder up, and run the following command in command-prompt:

"tensorboard --logdir=metviz_logs/"

If it finds the directory, it should create a visualization at some localhost port, and allow you to run various visualizations and simulations from the tensorboard interface.

### Sprite extracting code (taken from somewhere else)

The sprite is a file image that takes 28x28 px versions of each artworks and pastes them orderly in a grid. Always try to have an even number of images that can fit in complete rows and columns. Also make sure the order of the sprite thumbnails in the folder and the metadata file made in the next section coincide, as it may lead to a visualization showing incorrect thumbnails.

In [13]:
from PIL import Image
import os, math, time

In [14]:
max_frames_row = 31.0
frames = []
tile_width = 0
tile_height = 0

spritesheet_width = 0
spritesheet_height = 0

In [15]:
files = os.listdir("CroppedSprit/")
files.sort()
print(files)

['f00000_f00000_A Basket of Clams12388.jpg', 'f00001_f00001_A Bear Walking459184.jpg', 'f00002_f00002_A Descriptive Atlas of the Cesnola Collection of Cypriote Antiquities in the Metropolitan Museum of Art, New York  1885 1903 591850.jpg', 'f00003_f00003_A Gathering at Wood s Edge337105.jpg', 'f00004_f00004_A Giant Seated in a Landscape, sometimes called  The Colossus 334002.jpg', 'f00005_f00005_A Goldsmith in his Shop459052.jpg', 'f00006_f00006_A Gorge in the Mountains (Kauterskill Clove)10946.jpg', 'f00007_f00007_A Hunting Scene437283.jpg', 'f00008_f00008_A King Offers to Make Amends to a Bereaved Mother , Folio from a Khamsa  Quintet  of Amir Khusrau Dihlavi446560.jpg', 'f00009_f00009_A Knight of the d Aluye Family470599.jpg', 'f00010_f00010_A Muslim Pilgrim Learns a Lesson in Piety from a Brahman , Folio from a Khamsa  Quintet  of Amir Khusrau Dihlavi446563.jpg', 'f00011_f00011_A Rose14931.jpg', 'f00012_f00012_A Stallion453336.jpg', 'f00013_f00013_A Tusk Figurine of a Man547232.jpg

In [16]:
for current_file in files :
    try:
        with Image.open("CroppedSprit/" + current_file) as im :
            frames.append(im.getdata())
    except:
        print(current_file + " is not a valid image")

tile_width = frames[0].size[0]
tile_height = frames[0].size[1]

if len(frames) > max_frames_row :
    spritesheet_width = tile_width * max_frames_row
    required_rows = math.ceil(len(frames)/max_frames_row)
    spritesheet_height = tile_height * required_rows
else:
    spritesheet_width = tile_width*len(frames)
    spritesheet_height = tile_height
    
print(spritesheet_height)
print(spritesheet_width)

f00617_f00617_Proposition concernant le payement et la police des troupes du roy, qui produira   Sa Majest  une finance de six millions deux cens soixante mille livres invent e et propos e par le baron de Sparre, circa 1740681274.jpg is not a valid image
896
868.0


In [17]:
spritesheet = Image.new("RGBA",(int(spritesheet_width), int(spritesheet_height)))

for current_frame in frames :
    top = tile_height * math.floor((frames.index(current_frame))/max_frames_row)
    left = tile_width * (frames.index(current_frame) % max_frames_row)
    bottom = top + tile_height
    right = left + tile_width
    
    box = (left,top,right,bottom)
    box = [int(i) for i in box]
    cut_frame = current_frame.crop((0,0,tile_width,tile_height))
    
    spritesheet.paste(cut_frame, box)
    
spritesheet.save("spritesheet" + time.strftime("%Y-%m-%dT%H-%M-%S") + ".png", "PNG")

### Create a metadata file to relate index with filename

In [30]:
files[1][14:-4]

'A Bear Walking459184'

In [31]:
labels = []

for marquilla in files:
    labels.append(marquilla[14:-4])

In [ ]:
with open(embedding.metadata_path, 'w') as meta:
    meta.write('Index\tLabel\n')
    for index, label in enumerate(labels):
        meta.write('{}\t{}\n'.format(index, label))